# 1.优化器

深度学习优化算法大概经历了 SGD -> SGDM -> NAG ->Adagrad -> Adadelta(RMSprop) -> Adam -> Nadam 这样的发展历程。

详见《一个框架看懂优化算法之异同 SGD/AdaGrad/Adam》

https://zhuanlan.zhihu.com/p/32230623

对于一般新手炼丹师，优化器直接使用Adam，并使用其默认参数就OK了。

一些爱写论文的炼丹师由于追求评估指标效果，可能会偏爱前期使用Adam优化器快速下降，后期使用SGD并精调优化器参数得到更好的结果。

此外目前也有一些前沿的优化算法，据称效果比Adam更好，例如LazyAdam, Look-ahead, RAdam, Ranger等.

在keras.optimizers子模块中，它们基本上都有对应的类的实现。

* SGD, 默认参数为纯SGD, 设置momentum参数不为0实际上变成SGDM, 考虑了一阶动量, 设置 nesterov为True后变成NAG，即 Nesterov Acceleration Gradient，在计算梯度时计算的是向前走一步所在位置的梯度。

* Adagrad, 考虑了二阶动量，对于不同的参数有不同的学习率，即自适应学习率。缺点是学习率单调下降，可能后期学习速率过慢乃至提前停止学习。

* RMSprop, 考虑了二阶动量，对于不同的参数有不同的学习率，即自适应学习率，对Adagrad进行了优化，通过指数平滑只考虑一定窗口内的二阶动量。

* Adadelta, 考虑了二阶动量，与RMSprop类似，但是更加复杂一些，自适应性更强。

* Adam, 同时考虑了一阶动量和二阶动量，可以看成RMSprop上进一步考虑了Momentum。

* Nadam, 在Adam基础上进一步考虑了 Nesterov Acceleration。

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers,models,losses,metrics,callbacks
import tensorflow.keras.backend as K 

## 1.优化器的使用

优化器主要使用apply_gradients方法传入变量和对应的梯度从而来对给定变量进行迭代,或者直接使用minimize方法对目标函数进行迭代.

更常用的方法是在编译时将优化器传入keras的Model,通过调用model.fit实现对loss的迭代优化.

初始化优化器时会创建一个变量optimier.iterations用于记录迭代的次数,因此优化器和tf.Variable一样,一般需要@tf.function方法外创建.

In [2]:
#打印时间分割线
@tf.function
def printbar():
    ts = tf.timestamp()
    today_ts = ts%(24*60*60)

    hour = tf.cast(today_ts//3600+8,tf.int32)%tf.constant(24)
    minite = tf.cast((today_ts%3600)//60,tf.int32)
    second = tf.cast(tf.floor(today_ts%60),tf.int32)
    
    def timeformat(m):
        if tf.strings.length(tf.strings.format("{}",m))==1:
            return(tf.strings.format("0{}",m))
        else:
            return(tf.strings.format("{}",m))
    
    timestring = tf.strings.join([timeformat(hour),timeformat(minite),
                timeformat(second)],separator = ":")
    tf.print("=========="*8,end = "")
    tf.print(timestring)

In [5]:
## 求f(x)=a*x**2 + b * x + c的最小值
##使用optimizer.apply_gradients

x = tf.Variable(0.,name='x',dtype=tf.float32)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

@tf.function
def minimizef():
    a = tf.constant(1.)
    b = tf.constant(-2.)
    c = tf.constant(1.)
    
    while tf.constant(True):
        with tf.GradientTape() as tape:
            y = a*tf.pow(x,2) + b*x + c
        dy_dx = tape.gradient(y,x)
        optimizer.apply_gradients(grads_and_vars=[(dy_dx,x)])
        
        ##迭代终止条件
        if tf.abs(dy_dx) < tf.constant(0.00001):
            break
        
        if tf.math.mod(optimizer.iterations,100) == 0:
            printbar()
            tf.print('step=',optimizer.iterations)
            tf.print('x=',x)
    
    y = a*tf.pow(x,2) + b*x + c
    return y

In [6]:
tf.print('y=',minimizef(),'x=',x)

================================================================================22:08:35
step= 100
x= 0.867380381
================================================================================22:08:35
step= 200
x= 0.98241204
================================================================================22:08:35
step= 300
x= 0.997667611
================================================================================22:08:35
step= 400
x= 0.999690711
================================================================================22:08:35
step= 500
x= 0.999959
================================================================================22:08:35
step= 600
x= 0.999994516
y= 0 x= 0.999995232


In [7]:
def f():
    a = tf.constant(1.)
    b = tf.constant(-2.)
    c = tf.constant(1.)
    y = a*tf.pow(x,2) + b*x + c
    return y

@tf.function
def train(epoch=1000):
    for _ in tf.range(epoch):
        optimizer.minimize(f,[x])
    tf.print('epoch=',optimizer.iterations)
    return f()
train(1000)
tf.print('y=',f(),'x=',x)

epoch= 1606
y= 0 x= 0.999998569


In [8]:
## 使用model.fit

tf.keras.backend.clear_session()

class FakeModel(tf.keras.models.Model):
    def __init__(self,a,b,c):
        super(FakeModel,self).__init__()
        self.a = a
        self.b = b
        self.c = c
    
    def build(self):
        self.x = tf.Variable(0.,name='x',dtype=tf.float32)
        self.built = True
    
    def call(self,features):
        loss = self.a*(self.x)**2+self.b*(self.x)+self.c
        return(tf.ones_like(features)*loss)

def myloss(y_true,y_pred):
    return tf.reduce_sum(y_pred)

model = FakeModel(tf.constant(1.),tf.constant(-2.),tf.constant(1.))

model.build()
model.summary()

model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),loss=myloss)    
history=model.fit(tf.zeros((100,2)),tf.ones(100),batch_size=1,epochs=10)

tf.print('x=',model.x)
tf.print('loss=',model(tf.constant(0.)))

Model: "fake_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 1
Trainable params: 1
Non-trainable params: 0
_________________________________________________________________
Train on 100 samples
Epoch 1/10
100/100 [==============================] - 0s 1ms/sample - loss: 0.2550
Epoch 2/10
100/100 [==============================] - 0s 542us/sample - loss: 7.2587e-05
Epoch 3/10
100/100 [==============================] - 0s 451us/sample - loss: 1.5497e-08
Epoch 4/10
100/100 [==============================] - 0s 429us/sample - loss: 0.0000e+00
Epoch 5/10
100/100 [==============================] - 0s 422us/sample - loss: 0.0000e+00
Epoch 6/10
100/100 [==============================] - 0s 367us/sample - loss: 0.0000e+00
Epoch 7/10
100/100 [==============================] - 0s 370us/sample - loss: 0.0000e+00
Epoch 8/10
100/100 [==============================] - 0s 390us/sample - loss: 0.0000

# 2.回调函数callbacks

tf.keras的回调函数实际上是一个类，一般是在model.fit时作为参数指定，用于控制在训练过程开始或者在训练过程结束，在每个epoch训练开始或者训练结束，在每个batch训练开始或者训练结束时执行一些操作，例如收集一些日志信息，改变学习率等超参数，提前终止训练过程等等。

同样地，针对model.evaluate或者model.predict也可以指定callbacks参数，用于控制在评估或预测开始或者结束时，在每个batch开始或者结束时执行一些操作，但这种用法相对少见。

大部分时候，keras.callbacks子模块中定义的回调函数类已经足够使用了，如果有特定的需要，我们也可以通过对keras.callbacks.Callbacks实施子类化构造自定义的回调函数。

所有回调函数都继承至 keras.callbacks.Callbacks基类，拥有params和model这两个属性。

其中params 是一个dict，记录了 training parameters (eg. verbosity, batch size, number of epochs...).

model即当前关联的模型的引用。

此外，对于回调类中的一些方法如on_epoch_begin,on_batch_end，还会有一个输入参数logs, 提供有关当前epoch或者batch的一些信息，并能够记录计算结果，如果model.fit指定了多个回调函数类，这些logs变量将在这些回调函数类的同名函数间依顺序传递。

内置回调函数:

* BaseLogger： 收集每个epoch上metrics在各个batch上的平均值，对stateful_metrics参数中的带中间状态的指标直接拿最终值无需对各个batch平均，指标均值结果将添加到logs变量中。该回调函数被所有模型默认添加，且是第一个被添加的。

* History： 将BaseLogger计算的各个epoch的metrics结果记录到history这个dict变量中，并作为model.fit的返回值。该回调函数被所有模型默认添加，在BaseLogger之后被添加。

* EarlyStopping： 当被监控指标在设定的若干个epoch后没有提升，则提前终止训练。

* TensorBoard： 为Tensorboard可视化保存日志信息。支持评估指标，计算图，模型参数等的可视化。

* ModelCheckpoint： 在每个epoch后保存模型。

* ReduceLROnPlateau：如果监控指标在设定的若干个epoch后没有提升，则以一定的因子减少学习率。

* TerminateOnNaN：如果遇到loss为NaN，提前终止训练。

* LearningRateScheduler：学习率控制器。给定学习率lr和epoch的函数关系，根据该函数关系在每个epoch前调整学习率。

* CSVLogger：将每个epoch后的logs结果记录到CSV文件中。

* ProgbarLogger：将每个epoch后的logs结果打印到标准输出流中。

## 2.1 自定义回调函数

可以使用callbacks.LambdaCallBack编写较为简单的回调函数,也可以通过callbacks.Callback子类编写更为复杂的回调函数逻辑.

In [9]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers,models,losses,metrics,callbacks
import tensorflow.keras.backend as backend


In [10]:
## 使用LambdaCallBack编写较为简单的 回调函数
import json
json_log = open('./../data/keras_log.json',mode='wt',buffering=1)
json_logging_callback = callbacks.LambdaCallback(
    on_epoch_end=lambda epoch,logs: json_log.write(json.dumps(dict(epoch=epoch,**alogs))+'\n'),
    on_train_end = lambda logs:json_log.close()
)

In [11]:
## 自定义
class LearningRateScheduler(callbacks.Callback):
    
    def __int__(self, schedule, verbose=0):
        super(LearningRateScheduler,self).__int__()
        self.schedule = schedule
        self.verbose = verbose
    
    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer,'lr'):
            raise ValueError('Optimizer must have a "lr" attribure.')
        try:
            lr = float(backend.get_value(self.mdoel.optimizer.lr))
            lr = self.schedule(epoch,lr)
        except:
            lr = self.schedule(epoch)
        
        if not isinstance(lr,(tf.Tensor,float, np.float32, np.float64)):
            raise ValueError('The output of the "schedule" function should be float.')
        
        if isinstance(lr, ops.Tensor) and not lr.dtype.is_floating:
            raise ValueError('The dtype of Tensor should be float.')
        
        backend.set_value(self.model.optimizer.lr, backend.get_value(lr))
        
        if self.verbose > 0:
            print('\nEpoch %05d: LearningRateScheduler reducing learning rate to %s.' %(epoch+1,lr))
        
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = backend.get_value(self.model.optimizer.lr)
